In [1]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 5.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.6-cp311-cp311-linux_x86_64.whl size=168908 sha256=e2fa6f6b06e1d2f5b1c8e76532e81de5f3821c58f820685f1ef53d179ad3296b
  Stored in directory: /home/jovyan/.cache/pip/wheels/a2/c5/3d/b8fd44767c125a321b8d6d7fbcce0181d2f2d1e34967c81365
Successfully built psycopg2


In [3]:
import psycopg2

# PostgreSQLの情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# データベース操作など、必要なクエリを実行
# テーブル一覧を取得するクエリ
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""

# クエリを実行
cursor.execute(query)

# 結果を取得
tables = cursor.fetchall()

# 結果を表示
for table in tables:
    print(table[0])

# 接続を閉じる
conn.close()

geography_columns
geometry_columns
spatial_ref_sys


In [14]:
overpass_api_query = """
[out:json];
area["ISO3166-2"="JP-13"]->.a;
(
  way["leisure"="park"](area.a);
);
out geom;
"""

import requests

overpass_api_res = requests.post("http://overpass-api.de/api/interpreter" , data = overpass_api_query)
overpass_api_json = overpass_api_res.json()

parks = []

for element in overpass_api_json['elements']:
    if element['type'] == 'way' and 'geometry' in element:
        parks.append(element)

print(len(parks))

7697


In [17]:
# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# テーブルを作成
cursor.execute('CREATE TABLE parks (id BIGINT, name VARCHAR, name_en VARCHAR, geom GEOMETRY(Polygon, 4326));')

# 公園データを挿入
for park in parks:
    park_id = park['id']
    park_name = park['tags'].get('name', '').replace("'", "''")
    park_name_en = park['tags'].get('name:en', '').replace("'", "''")

    coordinates = park['geometry']

    if len(coordinates) < 3:
        continue

    # 座標データをPostGISの形式に変換
    polygon = 'POLYGON(('
    for coord in coordinates:
        lon = coord['lon']
        lat = coord['lat']
        polygon += f'{lon} {lat},'
    polygon = polygon.rstrip(',')  # 末尾のカンマを削除
    polygon += '))'

    # テーブルに挿入
    insert_query = f"INSERT INTO parks (id, name, name_en, geom) VALUES ({park_id}, '{park_name}', '{park_name_en}', ST_SetSRID(ST_GeomFromText('{polygon}'), 4326));"
    print(insert_query)
    cursor.execute(insert_query)

# コミットして変更を確定
conn.commit()

# 接続を閉じる
conn.close()

INSERT INTO parks (id, name, name_en, geom) VALUES (4848909, '旧芝離宮恩賜庭園', 'Former Shibarikyu Gardens', ST_SetSRID(ST_GeomFromText('POLYGON((139.7588683 35.6559528,139.7593745 35.6558768,139.7593736 35.6556537,139.7593739 35.6555947,139.7593753 35.6554168,139.75975 35.6553757,139.7595285 35.6536938,139.7573828 35.6539317,139.757418 35.6540817,139.7574938 35.6540708,139.7575309 35.6541788,139.7575792 35.6542508,139.7576167 35.654229,139.7577198 35.6543366,139.7575347 35.6544674,139.7579977 35.6562663,139.7581785 35.6562327,139.7581409 35.6560648,139.7588683 35.6559528))'), 4326));
INSERT INTO parks (id, name, name_en, geom) VALUES (7836744, '浜離宮恩賜庭園', 'Hamarikyu Gardens', ST_SetSRID(ST_GeomFromText('POLYGON((139.7597 35.6590754,139.7630612 35.6632516,139.7644248 35.6622712,139.7671256 35.6602288,139.7669984 35.6601232,139.7671787 35.6599913,139.7673104 35.6600891,139.7677004 35.6597943,139.7670173 35.6592801,139.7668676 35.6591685,139.7662824 35.6588593,139.7662287 35.6588958,139.7661774 

In [18]:
# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# 公園の数を取得
cursor.execute('SELECT COUNT(*) FROM parks;')
hospital_count = cursor.fetchone()[0]
print(f'公園の数: {hospital_count}')

# 一番広い公園の面積、ID、名前を取得
query = """
SELECT id, name, ST_Area(geom) AS area
FROM parks
ORDER BY area DESC
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchone()

# 結果を表示
park_id = result[0]
park_name = result[1]
park_area = result[2]
print(f"一番広い公園のID: {park_id}")
print(f"一番広い公園の名前: {park_name}")
print(f"一番広い公園の面積: {park_area} square meters")

# 接続を閉じる
conn.close()

公園の数: 7695
一番広い公園のID: 296118406
一番広い公園の名前: 柴又公園・江戸川グランド
一番広い公園の面積: 0.00015979797185491454 square meters
